In [1]:
PROJECT_ID = "lramsey-dev"
REGION = "us-central1"
import os
os.environ["PROJECT_ID"] = PROJECT_ID

In [2]:
import sys
sys.path.append("../../common/src")
sys.path.append("../src")

In [3]:
import tempfile
import os
from typing import List, Optional, Tuple, Dict
from common.utils.logging_handler import Logger
from common.models import (UserQuery, QueryResult, QueryEngine, QueryDocument,
                           QueryReference, QueryDocumentChunk, BatchJobModel)
from common.utils.errors import (ResourceNotFoundException,
                                 ValidationError)
from common.utils.http_exceptions import InternalServerError
from utils.errors import NoDocumentsIndexedException
from google.cloud import storage
from services import llm_generate, embeddings
from services.query import query_prompts
from services.query.vector_store import VectorStore
from services.query.data_source import DataSource
from config import (PROJECT_ID, DEFAULT_QUERY_CHAT_MODEL,
                        DEFAULT_QUERY_EMBEDDING_MODEL)
from services.query.query_service import query_generate

INFO: [config/config.py:57 - <module>()] Namespace File not found, setting job namespace as default
INFO: [config/config.py:107 - <module>()] ENABLE_GOOGLE_LLM = True
INFO: [config/config.py:108 - <module>()] ENABLE_OPENAI_LLM = True
INFO: [config/config.py:109 - <module>()] ENABLE_COHERE_LLM = True
INFO: [config/config.py:110 - <module>()] ENABLE_GOOGLE_MODEL_GARDEN = True
INFO: [config/config.py:111 - <module>()] ENABLE_TRUSS_LLAMA2 = True
INFO: [config/vector_store_config.py:40 - <module>()] Default vector store = [matching_engine]
INFO: [config/vector_store_config.py:49 - <module>()] PG_HOST = [127.0.0.1]
INFO: [config/vector_store_config.py:50 - <module>()] PG_DBNAME = [pgvector]
ERROR: [config/vector_store_config.py:77 - <module>()] Cannot connect to pgvector instance at 127.0.0.1: (psycopg2.OperationalError) connection to server at "127.0.0.1", port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?

(Background on this error at

In [5]:
storage_client = storage.Client(project=PROJECT_ID)

In [6]:
query_engine = "lukman-test-vs-8"
q_engine = QueryEngine.find_by_name(query_engine)
user_id = "epKVwPZeIBz1rrqcDm1B"

In [8]:
query_result = QueryResult.find_by_id("JoRKnorOeLFWMQdeOGo2")

In [9]:
query_references = [QueryReference.find_by_id(qrefid) for qrefid in query_result.query_refs]

In [11]:
query_reference_dicts = [
    ref.get_fields(reformat_datetime=True) for ref in query_references
]

In [12]:
query_reference_dicts

[{'query_engine_id': 'urjshigECiGqhj90MjJx',
  'query_engine': 'Test Vertex Search 8',
  'document_id': 'geHFIudyXMUN8ODpsJ64',
  'document_url': 'gs://lramsey-dev-downloads-test-vertex-search-8/cdl.html',
  'chunk_id': None,
  'document_text': '<b>CDL</b> Class and Restriction O Flowcharts <b>Commercial</b> Driver <b>License</b> Information (<b>CDL</b> 051) <b>Application</b> for <b>Commercial</b> Driving Privileges (<b>CDL</b> 002):English&nbsp;...',
  'created_time': '2024-03-15 20:30:07.627561+00:00',
  'last_modified_time': '2024-03-15 20:30:07.627561+00:00',
  'deleted_at_timestamp': None,
  'deleted_by': '',
  'archived_at_timestamp': None,
  'archived_by': '',
  'created_by': '',
  'last_modified_by': '',
  'id': '0v5lkqCehiBWXa4fZQLg'},
 {'query_engine_id': 'urjshigECiGqhj90MjJx',
  'query_engine': 'Test Vertex Search 8',
  'document_id': '8ulJkCSvpTsAAfNNgOWo',
  'document_url': 'gs://lramsey-dev-downloads-test-vertex-search-8/dmvforms.html',
  'chunk_id': None,
  'document_t

In [13]:
user_query = UserQuery.find_by_id("IBorI8Hxbs0ogTwT5JQF")

In [14]:
prompt = "How do I apply for a commercial license"
question_response = "I do not know the answer to this question."

In [15]:
user_query.update_history(prompt, question_response, query_reference_dicts)